# 1. 카테고리별 사기 주문 횟수

### 사기&사기가 아닌 것 카테고리별 주문 비율

In [ ]:
# 사기여부
fraudulent_data = data[data['Is Fraudulent'] == 1]
non_fraudulent_data = data[data['Is Fraudulent'] == 0]

# 비율
fraudulent_counts = fraudulent_data['Product Category'].value_counts(normalize=True) * 100
non_fraudulent_counts = non_fraudulent_data['Product Category'].value_counts(normalize=True) * 100

# 사기 그래프
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.pie(fraudulent_counts, labels=fraudulent_counts.index, autopct='%1.2f%%', startangle=140)
plt.title('Fraudulent Transactions by Product Category')

# 비사기 거래 그래프
plt.subplot(1, 2, 2)
plt.pie(non_fraudulent_counts, labels=non_fraudulent_counts.index, autopct='%1.2f%%', startangle=140)
plt.title('Non-Fraudulent Transactions by Product Category')

plt.tight_layout()
plt.show()

### 각각 사기&사기 아닌 것 주문 횟수

In [ ]:
plt.figure(figsize=(10, 6))

# 사기o 횟수
plt.subplot(1, 2, 1)
sns.countplot(x='Product Category', data=data[data['Is Fraudulent'] == 1], color='#FF8000')
plt.title('Fraudulent Transactions by Product Category')
plt.xlabel('Product Category')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.ylim(14200,15200)
for p in plt.gca().patches:
    plt.gca().annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

# 사기x 횟수
plt.subplot(1, 2, 2)
sns.countplot(x='Product Category', data=data[data['Is Fraudulent'] == 0])
plt.title('Non-Fraudulent Transactions by Product Category')
plt.xlabel('Product Category')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.ylim(270000,285000)
for p in plt.gca().patches:
    plt.gca().annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.tight_layout()
plt.show()


### 카테고리 내에서 사기인 물품과 아닌 물품의 비율

In [ ]:
#각 카테고리별 주문 비율 대비 사기o, 사기x 비율
plt.figure(figsize=(10, 6))
sns.countplot(x='Product Category', hue='Is Fraudulent', data=data)
plt.title('Product Category vs. Fraudulent Transactions')
plt.xlabel('Product Category')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.legend(title='Is Fraudulent', labels=['No', 'Yes'])


total_transactions = len(data)

for index, category in enumerate(data['Product Category'].unique()):
    no_count = len(data[(data['Product Category'] == category) & (data['Is Fraudulent'] == 0)])
    yes_count = len(data[(data['Product Category'] == category) & (data['Is Fraudulent'] == 1)])

    total_count = no_count + yes_count
    no_ratio = (no_count / total_count) * 100
    yes_ratio = (yes_count / total_count) * 100

    plt.text(index - 0.2, no_count, f'{no_ratio:.3f}%', color='black', ha='center', va='bottom')
    plt.text(index + 0.2, yes_count, f'{yes_ratio:.3f}%', color='black', ha='center', va='bottom')

plt.tight_layout()
plt.show()

### 카테고리별 전체 주문 대비 사기 주문 비율

In [ ]:
# 각 카테고리별로 사기 거래 비율 계산
fraud_ratio_by_category = data.groupby('Product Category')['Is Fraudulent'].mean() * 100


plt.figure(figsize=(10, 6))
sns.barplot(x=fraud_ratio_by_category.values, y=fraud_ratio_by_category.index, palette='Set2')
plt.title('Fraudulent Transaction Ratio by Product Category')
plt.xlabel('Fraudulent Transaction Ratio (%)')
plt.ylabel('Product Category')
plt.xlim(4.7,5.1)
plt.tight_layout()
plt.show()

### 카테고리별 사기 0 주문과 사기x 주문의 거래금액 사분위수

In [ ]:
data['Is Fraudulent'] = data['Is Fraudulent'].map({1: 'Yes', 0: 'No'})

# 박스플롯 시각화
plt.figure(figsize=(12, 8))
sns.boxplot(x='Transaction Amount', y='Product Category', hue='Is Fraudulent', data=data, palette={'Yes': 'red', 'No': 'blue'})
plt.title('Boxplot of Transaction Amount by Product Category')
plt.xlabel('Transaction Amount')
plt.ylabel('Product Category')
plt.tight_layout()
plt.show()

# 2. 주문수 * 주문량(quantity) -> 개수기준

### 사기 주문 물품 개수

In [ ]:
fraudulent_data = data[data['Is Fraudulent'] == 1]
non_fraudulent_data = data[data['Is Fraudulent'] == 0]

# 카테고리별 주문 수량 계산 (주문 수 * 주문량)
total_order_quantity_per_category_fraudulent = fraudulent_data.groupby('Product Category').apply(lambda x: (x['Quantity']).sum())
total_order_quantity_per_category_non_fraudulent = non_fraudulent_data.groupby('Product Category').apply(lambda x: (x['Quantity'] * x['Transaction Amount']).sum())


plt.figure(figsize=(12, 6))

# 사기일 때의 그래프
plt.subplot(1, 2, 1)
sns.barplot(x=total_order_quantity_per_category_fraudulent.index, y=total_order_quantity_per_category_fraudulent.values, color='#FF8000')
plt.title('Total Order Quantity by Product Category (Fraudulent)')
plt.xlabel('Product Category')
plt.ylabel('Total Order Quantity')
plt.xticks(rotation=45)
plt.ylim(325000,475000)



plt.tight_layout()
plt.show()

* quantity를 곱한것이 이전과 다른 결과를 보였기에 원래 한번에 주문하는 비율이 많은 카테고리인지 검사

### 개수 기준

In [ ]:
quantity_ratios = []
categories = []
quantities = []

for category in data['Product Category'].unique():
    category_data = data[data['Product Category'] == category]
    total_orders = len(category_data)
    quantity_counts = category_data['Quantity'].value_counts(normalize=True).sort_index()
    for i in range(1, 6):
        ratio = quantity_counts.get(i, 0) * 100
        quantity_ratios.append(ratio)
        categories.append(category)
        quantities.append(i)


df = pd.DataFrame({'Product Category': categories, 'Quantity': quantities, 'Quantity Ratio (%)': quantity_ratios})


plt.figure(figsize=(12, 6))
sns.barplot(x='Product Category', y='Quantity Ratio (%)', hue='Quantity', data=df)
plt.title('Quantity Ratio by Product Category')
plt.xlabel('Product Category')
plt.ylabel('Quantity Ratio (%)')
plt.xticks(rotation=45)
plt.legend(title='Quantity')
plt.ylim(19,20.5)
plt.tight_layout()
plt.show()

### 평균값 기준

In [ ]:
# 각 카테고리별로 평균 주문량 계산
average_order_quantity_per_category = data.groupby('Product Category')['Quantity'].mean().sort_values(ascending=False)

# 시각화
plt.figure(figsize=(12, 6))
sns.barplot(x=average_order_quantity_per_category.index, y=average_order_quantity_per_category.values, color='green')
plt.title('Average Order Quantity by Product Category')
plt.xlabel('Product Category')
plt.ylabel('Average Order Quantity')
plt.xticks(rotation=45)
plt.ylim(2.975,3.025)
plt.tight_layout()
plt.show()

# 3. 사기 주문과 그렇지 않은 주문 구매수량 사분위수, 비율 비교

### 사분위수

In [ ]:
#사기주문과 그렇지 않은 주문 사분위수 비교
plt.figure(figsize=(8, 6))
sns.boxplot(x='Is Fraudulent', y='Quantity', data=data)
plt.title('Comparison of Purchase Quantity between Fraudulent and Non-Fraudulent Transactions')
plt.xlabel('Is Fraudulent')
plt.ylabel('Quantity')

plt.tight_layout()
plt.show()

### min, max 계산

In [ ]:
min_quantity = data['Quantity'].min()
max_quantity = data['Quantity'].max()

print("Minimum Quantity:", min_quantity)
print("Maximum Quantity:", max_quantity)

### 사기주문과 구매량 사이의 연관성

In [ ]:
fraudulent_quantity_data = data[data['Is Fraudulent'] == 1]
non_fraudulent_quantity_data = data[data['Is Fraudulent'] == 0]

# 각각의 Quantity 값에 대한 사기 주문과 사기가 아닌 주문의 비율 계산
fraudulent_quantity_ratios = (fraudulent_quantity_data['Quantity'].value_counts(normalize=True) * 100).sort_index()
non_fraudulent_quantity_ratios = (non_fraudulent_quantity_data['Quantity'].value_counts(normalize=True) * 100).sort_index()

fraudulent_quantity_ratios_df = pd.DataFrame({
    'Quantity': fraudulent_quantity_ratios.index,
    'Ratio (%)': fraudulent_quantity_ratios.values,
    'Transaction Type': 'Fraudulent'
})

non_fraudulent_quantity_ratios_df = pd.DataFrame({
    'Quantity': non_fraudulent_quantity_ratios.index,
    'Ratio (%)': non_fraudulent_quantity_ratios.values,
    'Transaction Type': 'Non-Fraudulent'
})

# 사기 주문과 사기가 아닌 주문의 데이터프레임 합치기
combined_df = pd.concat([fraudulent_quantity_ratios_df, non_fraudulent_quantity_ratios_df])


plt.figure(figsize=(15, 6))
# 사기 주문에 대한 그래프
plt.subplot(1, 2, 1)
sns.barplot(x='Quantity', y='Ratio (%)', data=fraudulent_quantity_ratios_df, color='#FF8000')
plt.title('Quantity Ratio in Fraudulent Transactions')
plt.xlabel('Quantity')
plt.ylabel('Ratio (%)')
plt.ylim(18,21)

for index, value in enumerate(fraudulent_quantity_ratios.values):
    plt.text(index, value + 0.2, f'{value:.2f}%', ha='center', va='top')


plt.tight_layout()
plt.show()